In [1]:
# ABD matrices calculator based on classical laminate theory
# Example 4.2 
import numpy as np

In [2]:
# 定义函数由单层板的工程常数计算单层板折减刚度矩阵Q
def transform_engineeringConstants(E1, E2, v12, G12):
    # 根据工程常数计算柔度矩阵的系数
    S11 = 1/E1;
    S12 = -v12/E1;
    S22 = 1/E2;
    S66 = 1/G12;
    # 初始化柔度矩阵
    S = np.zeros((3,3))
    # 将柔度矩阵的非零元素依次赋值；注意Python的编码从0开始。
    S[0][0] = S11;
    S[0][1] = S12;
    S[1][0] = S[0][1];
    S[1][1] = S22;
    S[2][2] = S66;
    # 矩阵运算求解折减刚度矩阵
    Q = np.linalg.inv(S)
    return Q

In [3]:
# 定义函数由单层板折减刚度矩阵Q和铺层角度，计算单层板在全局坐标系下的折减刚度矩阵Q_bar
def transform_Q(Q, theta):
    # 计算转换矩阵 T
    c = np.cos(theta*np.pi/180.0)
    s = np.sin(theta*np.pi/180.0)
    T = np.zeros((3,3))
    T[0][0] = c**2;
    T[0][1] = s**2;
    T[0][2] = 2.0*c*s;
    T[1][0] = s**2;
    T[1][1] = c**2;
    T[1][2] = -2.0*c*s;
    T[2][0] = -s*c;
    T[2][1] = s*c;
    T[2][2] = c**2-s**2;
    # 定义R矩阵
    R = np.zeros((3,3))
    R[0][0] = 1;
    R[1][1] = 1;
    R[2][2] = 2;
    # 计算全局折减刚度矩阵
    Q_bar = np.linalg.multi_dot([np.linalg.inv(T), Q, R, T, np.linalg.inv(R)])
    return Q_bar

In [4]:
def height(thickness):
    # thickness = np.array([5, 5, 5])
    ply_num = len(thickness)
    h_total = np.sum(thickness[0:ply_num]) # 层合板总厚度
    height = np.zeros(ply_num+1)
    #Create array storing the heights of k plies about the midplane
    for k in range(0, ply_num+1):
        height[k]= - h_total/2.0 + np.sum(thickness[0:k])
    return height

In [5]:
def ABD_matrices(Q, statck, thickness):
    A = np.zeros([3, 3])
    B = np.zeros([3, 3])
    D = np.zeros([3, 3])
    h = height(thickness)
    for i in range(3):
        for j in range(3):
            for k in range(len(h)-1):
                Q_bar = transform_Q(Q, statck[k])
                ply_kA = h[k+1] - h[k]
                ply_kB = (h[k+1]**2 - h[k]**2)/2
                ply_kD = (h[k+1]**3 - h[k]**3)/3
                A[i,j] = A[i,j] + ply_kA * Q_bar[i,j]
                B[i,j] = B[i,j] + ply_kB * Q_bar[i,j]
                D[i,j] = D[i,j] + ply_kD * Q_bar[i,j]
    return (A, B, D)

In [6]:
Q = transform_engineeringConstants(181.0*1.0E9, 10.3*1.0E9, 0.28, 7.17*1.0E9) # Example 2.6
thickness = np.array([5, 5, 5])
statck = np.array([0, 30, -45])
ABD_matrices(Q, statck, thickness)
print("A = ", ABD_matrices(Q, statck, thickness)[0])
print("B = ", ABD_matrices(Q, statck, thickness)[1])
print("D = ", ABD_matrices(Q, statck, thickness)[2])

A =  [[ 1.73924086e+12  3.88386411e+11  5.66337309e+10]
 [ 3.88386411e+11  4.53253512e+11 -1.14063610e+11]
 [ 5.66337309e+10 -1.14063610e+11  4.52482544e+11]]
B =  [[-3.12883381e+12  9.85521554e+11 -1.07165613e+12]
 [ 9.85521554e+11  1.15779070e+12 -1.07165613e+12]
 [-1.07165613e+12 -1.07165613e+12  9.85521554e+11]]
D =  [[ 3.34320341e+13  6.46097724e+12 -5.24029369e+12]
 [ 6.46097724e+12  9.31977132e+12 -5.59591315e+12]
 [-5.24029369e+12 -5.59591315e+12  7.66277974e+12]]
